In [2]:
!pip install h2o
!pip install sklearn



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 177.0 MB 59 kB/s 
  Created wheel for h2o: filename=h2o-3.36.1.3-py2.py3-none-any.whl size=177103819 sha256=00b5637108c0a58639808570b79f974958b5df787f21d400e1d34a5747a0dab9
  Stored in directory: /root/.cache/pip/wheels/64/86/0f/d54be71f64b85d238edb037069bc866f2156cc563bf02be527
Successfully built h2o
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import pandas as pd
import h2o
from sklearn.preprocessing import LabelEncoder 
from h2o.automl import H2OAutoML
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,44 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.1.3
H2O_cluster_version_age:,11 days
H2O_cluster_name:,H2O_from_python_unknownUser_webeoi
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.172 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


In [7]:
from google.colab import drive
drive.mount('/content/drive')
PATH = '/content/drive/MyDrive/SkillFactory/DS_ML_data/'
file = 'mushrooms.csv'
df = pd.read_csv(PATH + file)
df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g


In [8]:
labelEncoder = LabelEncoder()
for column in df.columns:
    df[column] = labelEncoder.fit_transform(df[column])
df = h2o.H2OFrame(df)
df = df.asfactor()

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [9]:
# split data on Train/Test/Validation. Train 70% Test and Validation each is 15%
train_df, test_df = df.split_frame(ratios=[.7])

Для AutoML необходимо указать несколько обязательных параметров: x, y и training_frame. Здесь вы также можете настроить значения max_runtime_sec и max_models. 

Max_runtime_sec — обязательный параметр, а max_model — необязательный. 
По умолчанию все непереданные параметры принимают значение NULL. 
Параметр x — это вектор признаков из training_frame. Если вы не хотите использовать все признаки из переданного кадра, вы можете установить его, передав его в x.
На этих данных будем использовать все признаки (кроме таргета) и установим max_runtime_sec на 10 минут (по умолчанию некоторые из моделей могут занять много времени). Запустим AutoML:

In [10]:
y = "class"
train_columns = train_df.columns
train_columns.remove(y)

aml_model = H2OAutoML(max_runtime_secs=500, seed = 1)
aml_model.train(x = train_columns, y = y, training_frame = train_df)

AutoML progress: |
10:54:51.247: _train param, Dropping bad and constant columns: [veil-type]

████
10:55:11.568: _train param, Dropping bad and constant columns: [veil-type]

█
10:55:23.482: _train param, Dropping bad and constant columns: [veil-type]

████████████
10:57:00.226: _train param, Dropping unused columns: [veil-type]


10:57:02.816: _train param, Dropping bad and constant columns: [veil-type]

██
10:57:13.160: _train param, Dropping bad and constant columns: [veil-type]


10:57:16.504: _train param, Dropping bad and constant columns: [veil-type]

██████
10:58:07.750: _train param, Dropping bad and constant columns: [veil-type]

██████
10:58:54.45: _train param, Dropping bad and constant columns: [veil-type]

██████
10:59:40.110: _train param, Dropping unused columns: [veil-type]


10:59:41.830: _train param, Dropping unused columns: [veil-type]

█
10:59:49.209: _train param, Dropping bad and constant columns: [veil-type]

█
10:59:59.711: _train param, Dropping bad and cons

,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,737.0,737.0,264169.0,1.0,15.0,9.568521,2.0,37.0,23.769335




ModelMetricsBinomial: gbm
** Reported on train data. **

MSE: 1.2798858623734958e-34
RMSE: 1.1313204065928873e-17
LogLoss: 6.057182992499095e-19
Mean Per-Class Error: 0.0
AUC: 1.0
AUCPR: 1.0
Gini: 1.0

Confusion Matrix (Act/Pred) for max f1 @ threshold = 1.0: 


,,0,1,Error,Rate
0,0,2938.0,0.0,0.0,(0.0/2938.0)
1,1,0.0,2744.0,0.0,(0.0/2744.0)
2,Total,2938.0,2744.0,0.0,(0.0/5682.0)



Maximum Metrics: Maximum metrics at their respective thresholds


...


Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_pr_auc,training_lift,training_classification_error
0,,2022-07-20 10:56:45,1 min 21.734 sec,0.0,0.499708,0.692564,0.500000,0.482929,1.0000,0.517071
1,,2022-07-20 10:56:45,1 min 22.305 sec,5.0,0.307043,0.365539,0.999978,0.999976,2.0707,0.000528
2,,2022-07-20 10:56:45,1 min 22.462 sec,10.0,0.198187,0.215143,0.999961,0.999958,2.0707,0.002112
3,,2022-07-20 10:56:46,1 min 22.616 sec,15.0,0.136146,0.134601,0.999991,0.999990,2.0707,0.000704
4,,2022-07-20 10:56:46,1 min 22.766 sec,20.0,0.099366,0.087113,0.999993,0.999992,2.0707,0.000528
5,,2022-07-20 10:56:46,1 min 22.955 sec,25.0,0.073812,0.056801,0.999994,0.999994,2.0707,0.000528
6,,2022-07-20 10:56:46,1 min 23.123 sec,30.0,0.054557,0.037208,0.999998,0.999997,2.0707,0.000352
7,,2022-07-20 10:56:46,1 min 23.279 sec,35.0,0.042684,0.025120,1.000000,0.999999,2.0707,0.000352
8,,2022-07-20 10:56:46,1 min 23.431 sec,40.0,0.033737,0.016932,1.000000,1.000000,2.0707,0.000000
9,,2022-07-20 10:56:47,1 min 23.581 sec,45.0,0.025203,0.011307,1.000000,1.000000,2.0707,0.000000



See the whole table with table.as_data_frame()

Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,odor,6733.892578,1.000000,0.908688
1,spore-print-color,212.713638,0.031589,0.028704
2,gill-color,142.532578,0.021166,0.019234
3,population,90.445801,0.013431,0.012205
4,habitat,44.008080,0.006535,0.005939
5,cap-color,38.876888,0.005773,0.005246
6,ring-number,27.834269,0.004133,0.003756
7,gill-size,25.774132,0.003828,0.003478
8,ring-type,20.975292,0.003115,0.002830
9,stalk-root,19.644810,0.002917,0.002651



See the whole table with table.as_data_frame()


Мы задали значение параметра Max_runtime_sec ~10 минут, но также есть параметр, отвечающий не за время, а за максимальное количество исследованных моделей. Помимо этого, есть возможности дополнительной настройки процесса работы AutoML и множество дополнительных параметров, которые вы можете задать. 

https://docs.h2o.ai/h2o/latest-stable/h2o-docs/automl.html Подробнее о параметрах можно прочитать в официальной документации.

После запуска и обучения H2OAutoML вы можете посмотреть, какие модели в нашем случае сработали лучше всего, и выбрать их для дальнейшего исследования.



In [11]:
leaderboard = aml_model.leaderboard
leaderboard.head()

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
GBM_1_AutoML_1_20220720_105450,1,2.86954e-16,1,0,1.95132e-14,3.80764e-28
GBM_4_AutoML_1_20220720_105450,1,2.16886e-18,1,0,3.51947e-17,1.23867e-33
XGBoost_grid_1_AutoML_1_20220720_105450_model_3,1,0.00418738,1,0,0.0177472,0.000314962
StackedEnsemble_BestOfFamily_3_AutoML_1_20220720_105450,1,0.00081096,1,0,0.000884708,7.82708e-07
GBM_grid_1_AutoML_1_20220720_105450_model_1,1,4.45454e-11,1,0,9.91415e-10,9.82904e-19
StackedEnsemble_AllModels_1_AutoML_1_20220720_105450,1,0.000679646,1,0,0.000679876,4.62231e-07
GBM_3_AutoML_1_20220720_105450,1,1.99301e-18,1,0,3.24697e-17,1.05428e-33
GLM_1_AutoML_1_20220720_105450,1,0.0014195,1,0,0.00602156,3.62592e-05
XGBoost_3_AutoML_1_20220720_105450,1,0.00285881,1,0,0.0120301,0.000144724
StackedEnsemble_AllModels_2_AutoML_1_20220720_105450,1,0.00066847,1,0,0.000668679,4.47131e-07


Давайте проверим результат предсказания на тестовых данных и посмотрим, не переобучилась ли модель:

In [13]:
prediction = aml_model.predict(test_df)

gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%


In [14]:
from sklearn import metrics

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

predict,p0,p1
0,1,1e-19
0,1,1e-19
0,1,1e-19
0,1,1e-19
1,0,1
0,1,2.68651e-19
0,1,1e-19
0,1,1e-19
1,0,1
0,1,1e-19


Для наших данных H2O AutoML показал отличные результаты на тестовой выборке. Это значит, что модель не переобучилась (в случае переобучения на test мы бы имели низкую оценку, а на train — высокую).



Получив лучшую (из исследуемых) модель, можно выполнить несколько шагов, которые могут повысить производительность модели (на следующем этапе после MVP):

Подготовка признаков (feature engineering).
Добавление новых признаков с учётом имеющихся зависимостей и выявление новых (feature extraction).
Отбор наиболее важных признаков для модели (feature selection). 
Многие из лучших моделей в H2O AutoML используют ансамблирование. Эти модели можно дополнительно настроить.


✔️ Несмотря на то, что AutoML не принесёт вам призовых мест в соревнованиях по машинному обучению на Kaggle или лучших результатов для рабочих задач, его определённо стоит рассмотреть в качестве дополнения к вашим простым моделям и бейзлайну.

✔️  AutoML может обрабатывать множество различных типов наборов данных, включая бинарную классификацию (как мы посмотрели выше), многоклассовую классификацию, а также задачи регрессии.

✔️  AutoML — отличный инструмент, помогающий в работе (а не заменяющий её). Но когда дело доходит до feature engineering, такой автоматизированный инструмент не может превзойти творческое мышление человека. 

Статьи со ссылками на GitHub-репозитории в области машинного обучения

Крутые GitHub репозитории в области машинного обучения
https://habr.com/ru/post/584048/


Другой Github 2: машинное обучение, датасеты и Jupyter Notebooks
https://habr.com/ru/company/vk/blog/445530/


30 самых удивительных проектов по машинному обучению
https://rb.ru/story/30-ml-projects/


**Дополнительные источники с предобученными моделями**

Model Zoo (GitHub)
https://modelzoo.co/


facebookarchive
https://github.com/facebookarchive/models


github.com/BVLC/caffe/wiki/Model-Zoo
https://github.com/BVLC/caffe/wiki/Model-Zoo


Keras Applications
https://keras.io/api/applications/


Torchvision Models
https://pytorch.org/vision/0.8/models.html


TensorFlow Models
https://github.com/tensorflow/models